In [ ]:
import os
import openai
import requests
from gtts import gTTS
from moviepy.editor import *
from PIL import Image

In [ ]:
openai.api_key = "YOUR_API_KEY_HERE"

In [ ]:
story = input()

In [ ]:
os.makedirs("images", exist_ok=True)
os.makedirs("voices", exist_ok=True)
os.makedirs("output", exist_ok=True)

In [ ]:
#split into lines
scenes = [line.strip() for line in story.split("\n") if line.strip()]
print(f"Detected {len(scenes)} scenes.")

In [ ]:
#Generate Cartoon Prompts
prompts = []
for i, scene in enumerate(scenes, 1):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You generate short, vivid cartoon-style prompts for DALL·E based on a story scene."},
            {"role": "user", "content": f"Create a cute cartoon DALL·E prompt for this story scene: {scene}"}
        ],
        temperature=0.7
    )
    prompt = response['choices'][0]['message']['content']
    prompts.append(prompt)
    print(f"Scene {i} Prompt: {prompt}")

In [ ]:
# Generate images using Dall-E
image_files = []
for i, prompt in enumerate(prompts, 1):
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024"
    )
    image_url = image_response.data[0].url
    img_path = f"images/scene{i}.png"
    img_data = requests.get(image_url).content
    with open(img_path, "wb") as f:
        f.write(img_data)
    image_files.append(img_path)
    print(f"✅ Scene {i} image saved to {img_path}")

In [ ]:
#Voice Over using Google Text to Speech
voice_files = []
for i, scene in enumerate(scenes, 1):
    voice_path = f"voices/scene{i}.mp3"
    tts = gTTS(scene, lang='en')
    tts.save(voice_path)
    voice_files.append(voice_path)
    

In [ ]:
#Combining Audio and Video
clips = []
for img, voice in zip(image_files, voice_files):
    audio_clip = AudioFileClip(voice)
    duration = audio_clip.duration or 3.0
    img_clip = ImageClip(img).set_duration(duration).set_audio(audio_clip)
    clips.append(img_clip)


In [ ]:
final_video = concatenate_videoclips(clips, method="compose")

output_video_path = "output/cartoon_story_with_voice.mp4"
final_video.write_videofile(
    output_video_path,
    fps=24,
    codec='libx264',
    audio_codec='aac'
)